# **Setup trainning environment**

In [ ]:
import os

#######################################
# Update python
#######################################

#!apt install python3.8-dev

#######################################
# Install CUDA 11.1
#######################################

#!wget https://developer.download.nvidia.com/compute/cuda/11.1.0/local_installers/cuda_11.1.0_455.23.05_linux.run
#!sh cuda_11.1.0_455.23.05_linux.run

#######################################
# Clone Brazilian Portuguese TTS repository
#######################################

!git clone https://github.com/stivenschwanz/tts_pt_br_neutral.git
os.chdir('/content/tts_pt_br_neutral')

#######################################
# Clone Coqui AI TTS repository and patch it
#######################################

!git clone https://github.com/coqui-ai/TTS
!patch TTS/TTS/utils/audio.py tts_utils_audio_py.patch 

#######################################
# Install requeriments
#######################################

os.chdir('/content/tts_pt_br_neutral/TTS')
!apt install espeak
!pip install -r requirements.txt
!python setup.py install


# **Preprocessing Brazilian Portuguese corpus**

In [ ]:
#######################################
# Restore TTS checkpoints
#######################################

os.mkdir('/content/tts_pt_br_neutral/checkpoints')
os.chdir('/content/tts_pt_br_neutral/checkpoints')
!gdown --id 1WZdLA0yF35J1cQBvlTvSiLdru1pj4erI
!unzip tacotron2-ddc-agosto-20-2021_12+14-c3082267.zip

#######################################
# PT-BR neutral corpus
#######################################

os.chdir('/content/tts_pt_br_neutral/')
!gdown --id 1nf2GxGO5nrAprj2kGQGUXmedOjEMM0S5
!unzip pt_br_neutral_corpus.zip

#######################################
# Create train-val splits
#######################################

!shuf pt_br_neutral_corpus/metadata.csv > pt_br_neutral_corpus/metadata_shuf.csv
!head -n 6000 pt_br_neutral_corpus/metadata_shuf.csv > pt_br_neutral_corpus/metadata_train.csv
!tail -n 728 pt_br_neutral_corpus/metadata_shuf.csv > pt_br_neutral_corpus/metadata_val.csv

#######################################
# Compute dataset mean and variance for normalization
#######################################

os.chdir('/content/tts_pt_br_neutral/TTS')
!python TTS/bin/compute_statistics.py ../pt_br_neutral_model_config.json ../scale_stats.npy --data_path ../pt_br_neutral_corpus/ --output_path ../checkpoints

# **Train the Tacotron2 network using the Brazilian Portuguese corpus**

In [ ]:

#######################################
# Trainning. Note that this is an optional step to continue training and improve the best model so far....
#######################################

os.chdir('/content/tts_pt_br_neutral/TTS')
# change the GPU id if needed                                  
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
!python TTS/bin/train_tts.py --config_path ../pt_br_neutral_model_config.json \
                                                     --output_path ../checkpoints \
                                                     --coqpit.datasets.0.path ../pt_br_neutral_corpus/ \
                                                     --coqpit.audio.stats_path ../scale_stats.npy

# **Test Tacotron2 network**

In [ ]:
import IPython

#######################################
# Set parameters
#######################################

MODEL_PATH='../checkpoints/tacotron2-ddc-agosto-20-2021_12+14-c3082267/best_model.pth.tar'
MODEL_CONFIG_PATH='../checkpoints/tacotron2-ddc-agosto-20-2021_12+14-c3082267/config.json'

TEXT='bom dia! tá na hora de matar a fome! tá na mesa pessoal'
#TEXT="O corpo docente era constituído de vinte e nove membros"

#######################################
# Testing...
#######################################

os.chdir('/content/tts_pt_br_neutral/TTS')
!python TTS/bin/synthesize.py --text "$TEXT" --model_path $MODEL_PATH --config_path $MODEL_CONFIG_PATH 

#######################################
# Play audio
#######################################

IPython.display.Audio('tts_output.wav') 
